In [ ]:
import pandas as pd
#from google.colab import userdata
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import numpy as np
import concurrent.futures

load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = os.getenv("BASE_URL")
#print(API_KEY)

In [ ]:
VOCABULARY = """Бавовна – іронічне позначення вибухів на території Росії або окупованих територіях, яке виникло через цензуру в російських медіа. Замінює слово «вибух» у контексті ударів по ворожих об’єктах.

Двохсотий – військовий термін, що означає загиблого солдата (походить від радянської кодової назви «вантаж 200» для транспортування тіл загиблих).

Приліт – потрапляння ракети, снаряду або дрону в ціль, зазвичай супроводжується вибухом.

Трьохсотий – військовий термін, що означає пораненого солдата (походить від кодової назви «вантаж 300» для евакуації поранених).

Прилетіти – отримати влучання ракетою чи снарядом, зазвичай використовується щодо обстрілів міст, військових об’єктів або техніки.

Втомитися – евфемізм, яким часто описують стан російських систем ППО або техніки після удару ЗСУ.

Пташка – безпілотник або літальний апарат, який виконує розвідувальні чи ударні завдання.

Ціль – об’єкт, по якому планується завдати удару (наприклад, військова техніка, командний пункт, склад боєприпасів).

Спеціальна воєнна операція – евфемістичний термін, який Росія використовує для позначення свого повномасштабного вторгнення в Україну з метою уникнення слова «війна».

Щит – живий щит, тобто цивільні або мобілізовані, яких російські війська використовують для прикриття своїх позицій.

Приземлити – збити ворожий літак, безпілотник чи ракету.

Мопед – іронічна назва іранського дрона-камікадзе «Shahed», який використовується Росією для ударів по українській інфраструктурі (через характерний звук двигуна, схожий на мотор мопеда).

Батальйон Монако – саркастичний термін для українських багатіїв та політиків, які втекли за кордон під час війни, особливо в дорогі курортні місця на кшталт Монако.

Дискотека – масований обстріл або бомбардування, часто супроводжується вибухами та загравою.

Зоряні війни – протиповітряний бій із застосуванням ППО, коли в небі видно сліди від збитих ракет або дронів.

За рускім кораблем – скорочена форма українського військового мему «Русскій корабль, іді нах*й!», що став символом спротиву російській агресії.

Дружній вогонь – випадковий обстріл своїх військ або техніки, часто через погану координацію або паніку.

На концерт до Кобзона – евфемізм, який означає загибель російських військових чи командирів (Йосип Кобзон – радянський співак, що підтримував російську агресію, помер у 2018 році).

Мінусувати – знищувати ворожу техніку або живу силу (наприклад, «мінуснули танк» – знищили танк).

М’ясо – мобілізовані солдати, яких російське командування кидає в бій без належної підготовки та забезпечення (також відоме як «м’ясні штурми»).

Відпрацювати – завдати удару по ворожій позиції або техніці (наприклад, «артилерія відпрацювала по складу БК»).

Нуль – передова лінія фронту, найнебезпечніше місце, де тривають активні бойові дії.

На щиті – вираз, що означає загибель військового у бою. Походить із давньої традиції, коли загиблих воїнів приносили з поля бою на щитах. У сучасному контексті використовується як синонім терміна «двохсотий»."""

In [ ]:
WORD_LIST = "бавовна, двохсотий, приліт, трьохсотий, прилетіти, втомитися, пташка, ціль, спеціальна воєнна операція, щит, приземлити, мопед, Батальон Монако, дискотека, зоряні війни, за рускім кораблем, дружній вогонь, на концерт до Кобзона, мінусувати, м’ясо, відпрацювати, нуль"

In [ ]:
example_strings = []
example_labels = []

In [ ]:
def print_escaped_array(string_array):
    txt1 = '"'
    txt2 = '\\"'
    escaped_array = [f'"{s.replace(txt1, txt2)}"' for s in string_array]  # Escape quotes manually
    formatted_output = "" + ",\n".join(escaped_array) + ""  # Format as array
    return formatted_output

# Function to strip everything before '[' and after ']'
def strip_before_after_brackets(s):
    start = s.find("[")  # Find the first '['
    end = s.rfind("]")   # Find the last ']'

    if start != -1 and end != -1 and start < end:
        return s[start:end+1]  # Extract substring including brackets
    return ""  # Return empty string if brackets not found


def clean_strings(sa):
    return [str(s).strip('\n') for s in sa]

In [ ]:
def ai_request(string, model_name, query_context):
  client = OpenAI(api_key = API_KEY, base_url = BASE_URL)
  completion = client.chat.completions.create(
        model=model_name,
        store=True,
        messages=[
            {"role": "system", "content": query_context},
            {"role": "user", "content": string }
        ]
  )
  print(completion.choices[0].message.content, string)
  return completion.choices[0].message.content



def get_ai_results(messages, model, query_context):
  with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
      futures = {executor.submit(ai_request, message, model, query_context): message for message in messages}
      results = [future.result() for future in concurrent.futures.as_completed(futures)]

  return [r for r in results]


In [ ]:
#process set of strings using ChatGPT and
def process_with_ai(strings, labels, model_name, query_context):
  assert(len(strings)==len(labels))
  client = OpenAI(api_key = API_KEY, base_url = BASE_URL)

  array=[]
  replyes = []
  TOTAL = len(strings)
  i = 0
  while i<TOTAL:
    num = min(10, TOTAL-i)
    results = get_ai_results(strings[i:i+num], model_name, query_context)
    for r in results:
      array.append(1 if r[0]=='1' else 0)
      #print(r)
    replyes.extend(results)
    i+=num

  assert( len(array) == len(strings) )
  return (array, replyes)

In [ ]:
def process_sheet_by_llm(df : pd.DataFrame, model_name, query_context):
  num = min(5, df.shape[0])
  strings = clean_strings(df.loc[0:num, 'text'])
  labels = np.array( df.loc[0:num, 'label'], np.int8)
  print(len(strings))
  print(len(labels))
  (array, replyes) = process_with_ai(strings, labels, model_name, query_context)
  print(len(array))
  print(len(replyes))
  df.loc[0:num,'ai_label'] = array
  df.loc[0:num,'ai_reply'] = replyes
  #df.loc[0, 'text'] = "New Text"

In [ ]:
def process_with_model(model_name, query_context, exp_label):# Load the Excel file
  global example_labels
  global example_strings
  xls = pd.ExcelFile('PETs_Ukr.xlsx')

  # Display available sheet names
  print("Sheet names:", xls.sheet_names)

  dfs = pd.read_excel(xls, sheet_name=None)

  # Read and display all sheets
  for sheet in xls.sheet_names[0:]:
    if sheet=='Examples': continue

    #try:
    print(f"\nProcessing sheet: {sheet}")
    process_sheet_by_llm(dfs[sheet], model_name, query_context)
    #except:
    #  print(f"Error while processing {sheet}")
    #  return

  # Save all modified sheets back
  with pd.ExcelWriter(f'{exp_label}-Result-{model_name}.xlsx') as writer:
      for sheet_name, df in dfs.items():
          df.to_excel(writer, sheet_name=sheet_name, index=False)

  with open(f"{exp_label}.txt", 'w', encoding='utf-8') as f:
      f.write(str(query_context))
  

In [ ]:
def print_labels_statistics():# Load the Excel file
  xls = pd.ExcelFile('PETs_Ukr.xlsx')

  # Display available sheet names
  print("Sheet names:", xls.sheet_names)
  dfs = pd.read_excel(xls, sheet_name=None)

  total = (0,0)

  # Read and display all sheets
  for sheet in xls.sheet_names:
    if sheet=='Examples': continue

    df = dfs[sheet]
    num = df.shape[0]
    labels = np.array( df.loc[0:num, 'label'], np.int8)
    print(sheet, "->", len(labels), " positive=", np.sum(labels), " ratio = ", np.sum(labels)/len(labels))
    total = (total[0]+len(labels), total[1]+np.sum(labels))

  print("Total ->", total[0], " positive=", total[1], " ratio = ", total[1]/total[0])


print_labels_statistics()

In [ ]:
models = [
    #"deepseek-chat"
    #"o3-mini",
    #"o1-mini"
    #"gpt-4o",
    "gpt-4o-mini",
    #"gpt-4-turbo",
    #"gpt-4",
    #"gpt-3.5-turbo"
]

queries = [
        "Ти - лінгвіст, який має визначити, чи слово в кутових дужках є евфемізмом. "
        "Як відповідь напиши 1, якщо слово в кутових дужках використано, як евфемізм, і 0, якщо використане в звичному значенні. "
        "Також напиши один синонім або гіперонім до цього слова. ",

        "Ти - лінгвіст, який має визначити, чи слово в кутових дужках є евфемізмом. "
        "Як відповідь напиши 1, якщо слово в кутових дужках використано, як евфемізм, і 0, якщо використане в звичному значенні. "
        "Також напиши один синонім або гіперонім до цього слова. "
        "Вважай, що ці слова є евфемізмами в контексті війни: \n" + VOCABULARY
        #"Також напиши англійською значення цього слова в реченні"},
        #"Врахуй можливі значення слів у контексті україно-російської війни: \n" + VOCABULARY}, # voc2
        #"Вважай, що ці слова є евфемізмами в контексті війни: \n" + WORD_LIST}, #VOCABULARY},
        #"Вважай, що ці слова є евфемізмами в контексті війни: \n" + VOCABULARY},
]

exp_label = "0001"

for model_name in models:
  process_with_model(model_name, queries[0], exp_label)

print("DONE!!!")